In [37]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
import time
import pandas as pd
from selenium.common.exceptions import ElementClickInterceptedException,TimeoutException,NoAlertPresentException,NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC
import getData as GD
from getData import Extract_Data
from datetime import date
from datetime import datetime
import pyautogui
import pyperclip
import pandas as pd
import glob, os
import pickle
import pywinauto
import Actu_Colums as Ac
from rapidfuzz import process, utils
import re


# Funciones 

In [38]:
def write_texta(driver,Xpath,Text2wrt):
    texta = driver.find_element(By.XPATH,(Xpath))
    texta.clear()
    texta.send_keys(Text2wrt)

def swtich_wind(driver,Wind_name):
    ventanas_abiertas = driver.window_handles
    for ventana in ventanas_abiertas:
        # Cambia a la ventana
        driver.switch_to.window(ventana)
        # Comprueba si el título de la página es el que buscas
        if driver.title == Wind_name:  # reemplaza esto con el título de tu página
            break  # Si encontraste la ventana, sal del bucle    

def añadir_linea(nombre_archivo, linea):
    with open(nombre_archivo, 'a') as archivo:
        archivo.write(linea + '\n')

# Croe funcion que abre el panel y busca el caracter a seleccionar deacuerdo al texto ingresado
def selecb(driver,Menu,Valor):
    txtbox_EmprA = driver.find_element(By.XPATH,f'//*[@id="WIN_0_10000515"]/div/a[img[@alt="{Menu}"]]')
    txtbox_EmprA.click()
    selec = wait.until(EC.visibility_of_element_located((By.XPATH, f'//td[@class="MenuEntryName" and text()="{Valor}"]'))) 
    selec.click()
    time.sleep(1)
def Selec_M(N_iteracion,Lv_list):
    for i in range(0,N_iteracion):
        M_d_lv1 = wait.until(EC.visibility_of_element_located((By.XPATH,f"//td[text() = '{Lv_list[i]}']")))
        M_d_lv1.click()
def convertir_formato_tiempo(tiempo):
    # Determinar si el tiempo está en formato de 24 horas
    if "PM" or "pm" in tiempo and int(tiempo.split(":")[0]) > 12:
        formato = "%H:%M%p"
    else:
        formato = "%I:%M%p"
    
    # Convertir la cadena de tiempo al formato correcto
    tiempo_24h = datetime.strptime(tiempo, formato)
    
    # Convertir el tiempo al formato deseado
    tiempo_nuevo_formato = tiempo_24h.strftime("%H:%M %p")
    
    return tiempo_nuevo_formato

def make_fechaH(Fecha, Hora):
    try:
        Hora = int(Hora)
        fecha_inicio = Fecha.date()
        if isinstance(Hora, int): #si hora es un str
            #Hora = Hora.replace(' ','') # quito espacios
            #Hora = convertir_formato_tiempo(Hora) 
            # Convierte la cadena de texto a un objeto datetime
            hora_datetime = datetime.strptime(str(Hora), "%H")
            # Extrae el objeto time del objeto datetime
            hora_inicio = hora_datetime.time()
        else:
            hora_dt = datetime.strptime(str(hora), "%H")
            print(hora_dt.time())
            hora_inicio = hora_dt
        fecha_hora_inicio = datetime.combine(fecha_inicio, hora_inicio)
        fecha_hora_str = fecha_hora_inicio.strftime('%d-%m-%Y %H:%M:%S')
    except ValueError as e:
        print("El formato de la hora no es correcto. Debe ser 'HH:MM AM/PM'.",str(e))
        return None
    return fecha_hora_str
def contiene_numero(cadena):
    return any(caracter.isdigit() for caracter in cadena)
def nuevo_cambio(wait,driver):
    Apps_bttn = wait.until(EC.visibility_of_element_located((By.XPATH, "//div[img[@title='Mostrar lista de aplicaciones']]")))
    Apps_bttn.click()
    
    bar =  wait.until(EC.visibility_of_element_located((By.XPATH,'//*[@id="WIN_0_304248710"]/fieldset/div/dl')))
    bar.click()
    
    Ges_Cambs =  wait.until(EC.visibility_of_element_located((By.XPATH,"//*[@id='WIN_0_80077']/fieldset/div/div/div/div[a[span[text() = 'Gestión de cambios']]]")))
    Ges_Cambs.click()                                 
    
    Nuv_Cam =  wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="FormContainer"]/div[5]/div/div[10]/div/div[2]')))
    
    actions = ActionChains(driver)
    actions.move_to_element(Ges_Cambs)
    actions.click(Nuv_Cam)
    actions.perform()

#Relleno campos 
##Escribo en la primera pantalla
def fill_data(driver,wait,Resumen:str,Data:dict,oYm:str):
    wait.until(EC.presence_of_element_located((By.XPATH,"//div[label[text()= 'Servicio+']]//textarea")))
    M_d = wait.until(EC.element_to_be_clickable((By.XPATH,"//div[label[text() = 'Servicio+']]//a"))) #abro el menu desplegable
    M_d.click() 
    Selec_M(2,['RED - OPER Y CAL SERV  CAMPO','Energía']) 
    time.sleep(1)
    
    write_texta(driver,"//div[label[text() = 'Resumen*']]//textarea",Resumen)
    write_texta(driver,"//fieldset/div[label[text() = 'Notas']]//textarea",Resumen)
    M_d = wait.until(EC.element_to_be_clickable((By.XPATH,"//fieldset/div[label[text()='Grupo de gestores*+']]//a")))#abro el menu desplegable
    M_d.click() 
    
    
    Selec_M(3,['América Móvil Perú SAC','REGIONAL DEL NORTE',f"Emergencia Norte {Data['Zona de Trabajo:']}"]) ## Esto deberia reemplzar a lo de arriba 
    M_d1 = wait.until(EC.element_to_be_clickable((By.XPATH,"//fieldset/div[label[text()='Gestor de cambios+']]//a")))
    M_d1.click() 
    
    M_d_lv1 = wait.until(EC.element_to_be_clickable((By.XPATH,f"//tr[td[text() = '{oYm}']]")))
    M_d_lv1.click()
#################################################################################
def categorizar(wait,driver,Cat_op_lv1:str,Cat_op_lv2:str,Cat_op_lv3:str):
    time.sleep(2)###########CAtegorizacion#########################3
    cttg_bttn = '//a[text()="Categorización"]'
    Catego = wait.until(EC.element_to_be_clickable((By.XPATH, cttg_bttn)))
    Catego.click()
    wait.until(EC.presence_of_element_located((By.XPATH,cttg_bttn)))
    write_texta(driver,"//div[label[text() = 'Nivel 1 de categorización operativa+']]//textarea",Cat_op_lv1)
    write_texta(driver,"//div[label[text() = 'Nivel 2 de categorización operativa']]//textarea",Cat_op_lv2)
    write_texta(driver,"//div[label[text() = 'Nivel 3 de categorización operativa']]//textarea",Cat_op_lv3)
def Consola_d_Cambios(wait,driver):
    ################### COnsola de Gestion de cambios ######################################################
    Apps_bttn = wait.until(EC.visibility_of_element_located((By.XPATH, "//div[img[@title='Mostrar lista de aplicaciones']]")))
    Apps_bttn.click()
    
    bar =  wait.until(EC.visibility_of_element_located((By.XPATH,'//*[@id="WIN_0_304248710"]/fieldset/div/dl')))
    bar.click()
    
    Ges_Cambs =  wait.until(EC.visibility_of_element_located((By.XPATH,"//*[@id='WIN_0_80077']/fieldset/div/div/div/div[a[span[text() = 'Gestión de cambios']]]")))
    Ges_Cambs.click()                                 
    
    C_GCambio =  wait.until(EC.visibility_of_element_located((By.XPATH, "//span[@class='navLabel lvl1 ' and text()='Consola de Gestión de cambios']")))
    
    actions = ActionChains(driver)
    actions.move_to_element(Ges_Cambs)
    actions.click(C_GCambio)
    actions.perform()
def load_df_by_name(directorios:str, cadena:str) -> pd.DataFrame:
    df_list = []
    for dictory in directorios:
        ruta = Ac.buscar_archivo_mas_antiguo(dictory, cadena)
        try:
            df = pd.read_excel(ruta, sheet_name='Hoja2')
            #print('Machea con nombre de hoja2')
        except Exception as e:
            print(f"Error: {e}. Trying with 'Sheet1'")
            df = pd.read_excel(ruta, sheet_name='Sheet1')
            #print('Machea con nombre de Sheet1')
        print(ruta)
        df_list.append(df)
    return df_list

def Get_DataForms(Site_Code: str, ultimas_columnas: pd.DataFrame, Date: str, name_site):
    try:
        print(Site_Code)
        Data = ultimas_columnas[ultimas_columnas['Codigo del SITE'].str.contains(Site_Code[-4:], case=False) 
                                & (ultimas_columnas['Fecha de inicio:'] == pd.to_datetime(Date))].iloc[0].to_dict()
    except IndexError:
        print("Alerta: No hay fila coincidente para la variable 'Fecha de inicio:' o 'Codigo del SITE'")
        try:
            Data = ultimas_columnas[ultimas_columnas.Resumen.str.contains(name_site, case=False)].iloc[-1].to_dict()
        except IndexError:
            print("Alerta: No hay fila coincidente para la variable 'name_site'")
            Data = None
    return Data

def Get_PathMOP(archivos_ordenados:list[str], name_site:str):
    name_site = name_site.replace('_','')
    nombres_archivos = [os.path.basename(archivo) for archivo in archivos_ordenados]
    rutas_formateadas = [archivo.replace("'", "").replace("\\\\", "\\") for archivo in archivos_ordenados] 
    lista_limpia = [re.sub('[^a-zA-Z]', '', valor) for valor in nombres_archivos]
    series_a_eliminar = ['CRQ', 'xlsx', 'MOP', 'Solicitud']
    for serie in series_a_eliminar:
        lista_limpia = [valor.replace(serie, '') for valor in lista_limpia]
    resultado_MOP = process.extractOne(name_site, lista_limpia,processor=utils.default_process,score_cutoff=80)
    display(resultado_MOP)
    index_lista_RMOPS = resultado_MOP[2]
    ruta_MOP = rutas_formateadas[index_lista_RMOPS]
    #@TODO Necesito apicar la coincidencia parcial para buscar las rutas 
    return ruta_MOP

@TODO 
Busacar por nombre, Fecha dentro de la carpeta de MPOS el archivo y devolver la ruta,

Buscar dentro del excel del forms el nombre del site 

In [39]:
site_data= pd.read_csv(r'D:\Scripts1\Code\ActPEA\CODE\Temps\BD_Sites.csv')
site_data_N = site_data[site_data['Id.SIte'].str.startswith(('T','SAD','CL','CAC'))].copy()
site_data_N.drop([6826],axis=0,inplace=True)

In [40]:
# Especifica el path y el tipo de archivo
path_o = r'C:\Users\C26764\OneDrive - America Movil Peru S.A.C\*'
tipo_de_archivo = '*.xlsx'
archivos = glob.glob(path_o + tipo_de_archivo)
# Busca el archivo más reciente
#archivos
archivo_mas_reciente = max(archivos, key=os.path.getmtime)
nombre_del_archivo = os.path.basename(archivo_mas_reciente)
print(archivo_mas_reciente)
Crqs_path= archivo_mas_reciente
CRQ = pd.read_excel(Crqs_path, sheet_name='Form1') 
column_names = CRQ.columns
ultimas_columnas = CRQ[column_names[-7:]].copy()
ultimas_columnas.rename(columns={'Resumen de Actividad (hasta 50 palabras, incluir nombre del SITE):':'Resumen',
                                 'Código del SITE (TXNNNN)\n\n':'Codigo del SITE',
                                 'Hora de Inicio(hh:mm):':'Hora de Inicio',
                                 'Hora de Fin(hh:mm):':'Hora de Fin'},inplace=True)
#################################### MOP ##########################
## Cargo Archivo MOP
# Especifica el path y el tipo de archivo
path_o = r'C:\Users\C26764\America Movil Peru S.A.C\EAS - 2\MOP_CRQS\*'
tipo_de_archivo = '*.doc'
archivos = glob.glob(path_o)
# Ordena los archivos por fecha de modificación (el más reciente primero)
archivos_ordenados = sorted(archivos, key=os.path.getmtime, reverse=True)

C:\Users\C26764\OneDrive - America Movil Peru S.A.C\Solicitud de CRQ.xlsx


## Diccionario para cambiar code del Site por uno del REMEDY

In [45]:
Dic_RM = {'SAD0004':'TJ5254', 'SAD0010':'PDICAJAMARCA_100G_OSN9800U16','TT6492':'TTL64921','TL8307':'POP_PDI_CHICLAYO','TA2154':'TAF8196',
         'TP8713':'TMETROPIURAPROVINCIA','TA5008':'RCSRSANTASANTA'}


# Login

In [55]:
############ Me Logeo al remedy ###################
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 7)
driver.get("http://172.17.51.6:8082/arsys/shared/login.jsp?/arsys/")
GD.login(driver,user_ID="username-id",password_ID='pwd-id',user_NAME='C16985',password='Julio2024***')
#elementos = driver.find_elements(By.XPATH, '//*[@id="title_0"]/div') Claro2023*****
#######Switch to alert########### if alert present in driver: recargo la pagina 
try:
    alerta = WebDriverWait(driver, 5).until(EC.alert_is_present())
    print("Alerta detectada!")
    alerta.accept()  # o puedes usar alerta.dismiss() para cerrar la alerta
except TimeoutException:
    print("No se detectó ninguna alerta en el tiempo especificado.")
    try:
        Iframe = driver.find_element(By.XPATH, "//tr/td/iframe[@src='http://172.17.51.6:8082/arsys/resources/html/MessagePopup.html']") 
        #driver.switch_to.frame(Iframe)
        #bloqueado = driver.find_element(By.XPATH,('//*[@id="PopupMsgBox"]'))     
        driver.close()
        print("Pagina Bloqueada")
    except NoSuchElementException:
        pass
wait = WebDriverWait(driver, 10) 


Alerta detectada!


## Variables internas del CRQ

In [56]:
ultimas_columnas.tail(20)

,Codigo del SITE,Resumen,Zona de Trabajo:,Fecha de inicio:,Hora de Inicio,Fecha de Fin:,Hora de Fin
292,TL5943,ENERGIA DC - INSTALACION DE BATERIAS DC - LAMB...,Chiclayo,2024-07-18,8,2024-07-18,18
293,TL5943,ENERGIA DC - INSTALACION DE BATERIAS DC - LAMB...,Chiclayo,2024-07-19,8,2024-07-19,18
294,TA6475,Acarreo de materiales para fabricación de mate...,Huaraz,2024-07-17,8,2024-07-17,18
295,TT6492,BALANCEO DE RECTIFICADOR DE MONOFASICO A TRIFA...,Tumbes,2024-07-17,9,2024-07-17,20
296,TT6492,BALANCEO DE RECTIFICADOR DE MONOFASICO A TRIFA...,Tumbes,2024-07-17,9,2024-07-17,20
297,TJ5255,INSPECCION Y RETIRO DE GE,Trujillo,2024-07-18,10,2024-07-18,18
298,TA5655,VISITA TECNICA - TSS EL PINAR,Huaraz,2024-07-19,8,2024-07-19,18
299,TA5655,VISITA TECNICA - TSS EL PINAR,Huaraz,2024-07-19,8,2024-07-20,18
300,TA2154,VISITA TECNICA - TSS CONOCOCHA,Huaraz,2024-07-20,8,2024-07-20,18
301,TA2154,VISITA TECNICA - TSS CONOCOCHA,Huaraz,2024-07-21,8,2024-07-21,18


In [57]:
site_data_N[site_data_N['Id.SIte'] == 'TT6492']

,Id.SIte,SIte
2508,TT6492,FERNANDO_BELAUNDE_DE_TUMBES


In [58]:
name_site = input("Escribe el nombre del Site")
fecha_CRQ = input("Fecha de CRQ")
name_site = name_site.replace(' ','')

lista = site_data_N.SIte.tolist()
name_ofD = process.extractOne(name_site, lista,processor=utils.default_process)
#name_ofD = process.extractOne(name_site, lista)
print(name_ofD[0])
## Implementar para cuadno haya duplicados te pida el codigo.
print(f'Coincidencia al {name_ofD[1]} % de coincidencia')

#####Añdir seleccion para cuadno haya duplicado
Site_Code = site_data_N[site_data_N.SIte == name_ofD[0]]['Id.SIte'].iloc[0]
#Site_Code = 'TJ5083'
print(Site_Code)
Data = Get_DataForms(Site_Code,ultimas_columnas,fecha_CRQ,name_site)
Site_Code = Dic_RM.get(Site_Code,Site_Code)
print(Site_Code)

resultado = Get_PathMOP(archivos_ordenados,name_site)
display(Data)

print(resultado) 

Escribe el nombre del Site SANTA
Fecha de CRQ 2024-07-19


SANTA
Coincidencia al 100.0 % de coincidencia
TA5008
TA5008
RCSRSANTASANTA


('SANTASOLICITUD', 90.0, 0)

{'Codigo del SITE': 'TA5008',
 'Resumen': 'Acarreo de materiales, fabricación de losa de concreto, montaje de gabinete TX',
 'Zona de Trabajo:': 'Chimbote',
 'Fecha de inicio:': Timestamp('2024-07-19 00:00:00'),
 'Hora de Inicio': '8',
 'Fecha de Fin:': Timestamp('2024-07-22 00:00:00'),
 'Hora de Fin': '18'}

C:\Users\C26764\America Movil Peru S.A.C\EAS - 2\MOP_CRQS\SANTA SOLICITUD CRQ 20.07.xlsx


## Cargo oym

In [59]:
Data['Resumen'] = Data['Resumen'][:180]
Data['Codigo del SITE'] = Site_Code
#Ruta_MOP = r'C:\Users\C26764\America Movil Peru S.A.C\EAS - 2\MOP_CRQS\FORMATO CRQ 15-05 Moche.xlsx'
Ruta_MOP = resultado
######################################################################################################
## Diccionario de OyM
Dic_OyM = { 'Piura' : 'JUAN J PELAEZ HIGUARO' ,
            'Tumbes': 'JOSE E HONORES BARRERA',
            'Cajamarca':'WALTER J AGUILAR GARCIA' ,
            'Chachapoyas':'MIGUEL CHAVEZ PINEDO',
            'Chiclayo' :'VICTOR O GONZALES GUEVARA',
            'Chimbote' :'MANUEL J VALERIANO MENDOZA',
            'Chota'    : 'WALTER J AGUILAR GARCIA',
            'Huaraz'   :'HENTZ D BENAVIDES PAREDES',
            'Jaen'     : 'MIGUEL CHAVEZ PINEDO',
            'Trujillo' : 'LUCIANO CARRASCO ACARO', }
Resumen = Data['Resumen']
Resumen = Resumen.replace('\t',' ')
oYm = Dic_OyM[Data['Zona de Trabajo:']]
Lugar = 'Emergencia Norte '+ Data['Zona de Trabajo:']
print('OyM: ',oYm,Lugar)
## Fechas Formateadas 
FStar = make_fechaH(Data['Fecha de inicio:'],Data['Hora de Inicio'])
FEnd =  make_fechaH(Data['Fecha de Fin:'],Data['Hora de Fin'])

OyM:  MANUEL J VALERIANO MENDOZA Emergencia Norte Chimbote


# Para volver a la base para nueva CRQ
> - Bucle While para automatizar mas la creacion 

In [60]:
swtich_wind(driver,'BMC Remedy (Buscar)')
# Cambia de nuevo el enfoque a la pestaña original (la primera pestaña)
driver.switch_to.window(driver.window_handles[0])
driver.refresh()
try:
    # Cambiar a la alerta
    alert = driver.switch_to.alert

    # Aceptar la alerta
    alert.accept()
except NoAlertPresentException:
    print("No hay ninguna alerta presente.")
nuevo_cambio(wait,driver)
time.sleep(1)
fill_data(driver,wait,Resumen,Data,oYm)
CRQ = wait.until(EC.visibility_of_element_located((By.XPATH,'//*[@id="WIN_0_304248710"]/fieldset/div/dl/dd[5]/span[2]/a'))) # Forma de encontrar CRQ de un textarea 
CRQ = CRQ.text[:15]
CRQ

No hay ninguna alerta presente.


'CRQ000002863048'

In [61]:
Cat_op_lv1= 'SISTEMA DE ENERGÍA'
Cat_op_lv2='OPTIMIZACIÓN'
Cat_op_lv3 = 'SIN AFECTACIÓN'
categorizar(wait,driver,Cat_op_lv1,Cat_op_lv2,Cat_op_lv3)
###################Tareas#####################################
tareas_bttn = wait.until(EC.element_to_be_clickable((By.XPATH,'//div/div/div/dl/dd/span[a[text()="Tareas"]]')))
tareas_bttn.click()
#################3 Relaciono Tareas 
#wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="WIN_0_304316340"]/div')))
Rela = wait.until(EC.element_to_be_clickable((By.XPATH, "//fieldset/div/a[div[div[text()='Relacionar']]]")))
Rela.click()
time.sleep(2)

### Switcheo ventanas 

## @TODO: eL TIEMPO DE CARGA DEL ARCHIVO MOP ES VAIRABLE 

In [62]:
swtich_wind(driver,'Crear tarea')
########### Relleno valores #############3 
Resumen = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="arid_WIN_0_8"]')))  ########
content = Resumen.get_attribute('value')
write_texta(driver,"//*[@id='FormContainer']/div[label[text() = 'Nombre*']]//textarea",content)
write_texta(driver,"//*[@id='FormContainer']/div[label[text() = 'Notas']]//textarea",content)

### Agrego archivo adjuto

Agregar = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="WIN_0_10010508"]/div[3]/table/tbody/tr/td[1]/a[1]'))) 
Agregar.click() 
#Switcheo de iframe
time.sleep(1)
Iframe = wait.until(EC.presence_of_element_located((By.XPATH, "//tr/td/iframe[@src='http://172.17.51.6:8082/arsys/resources/html/AttachmentPopup.html']"))) 
driver.switch_to.frame(Iframe)
Cargar_arch = driver.find_element(By.XPATH, '//*[@id="PopupAttFileTable"]') #
Cargar_arch.click()

time.sleep(1)

### Necesitas enfocar la ventana donde se carga el archivo

#ruta = r'D:\MOPS\03.05 MOP Procedimiento Replanteo AA - PDI Chiclayo.doc'  # Escribe la ruta al archivo MOP
app = pywinauto.Application().connect(title='BMC Remedy (Buscar) - Google Chrome')
# Busca la ventana por su nombre
ventana = app.window(title='Crear tarea - Google Chrome')
ventana.set_focus()
# Trae la ventana al frente
rect = ventana.rectangle()

# Mueve el cursor a una posición dentro de la ventana
pyautogui.moveTo(rect.left + 100, rect.top + 100)

# Hace clic en la posición actual del cursor
pyautogui.click()
pyperclip.copy(Ruta_MOP)
time.sleep(0.5)
pyautogui.hotkey('ctrl', 'v')
pyautogui.press('enter')
time.sleep(2.5)

Aceptar = driver.find_element(By.XPATH, '//*[@id="PopupAttFooter"]/a[1]')
Aceptar.click()

###Volver a enfocar el opera

In [63]:


# click en categorizacion
swtich_wind(driver,'Crear tarea')
categ = wait.until(EC.element_to_be_clickable((By.XPATH,'//*[@id="WIN_0_10000500"]/div[2]/div[2]/div/dl/dd[span[a[contains(text(),"Categorización")]]]//span[2]')))
categ.click()
wait.until(EC.presence_of_element_located((By.XPATH,'//*[@id="WIN_0_10000518"]/div/textarea[@armenu="CFG:CTL:GLOBAL-S1-Q"]')))
time.sleep(1)
write_texta(driver,'//*[@id="WIN_0_10000518"]/div/textarea[@armenu="CFG:CTL:GLOBAL-S1-Q"]',Cat_op_lv1)
write_texta(driver,'//*[@id="WIN_0_10000518"]/div/textarea[@armenu="CFG:CTL:GLOBAL-S2-Q"]',Cat_op_lv2)
write_texta(driver,'//*[@id="WIN_0_10000518"]/div/textarea[@armenu="CFG:CTL:GLOBAL-S3-Q"]',Cat_op_lv3)

#TODO Añadir espera implicita para cargar el archivo
#################################################################################################################################################

        


Asig_Fech = driver.find_element(By.XPATH,'//*[@id="WIN_0_10000500"]/div[2]/div[2]/div/dl/dd[span[a[contains(text(),"Asignación/Fechas")]]]//a')
Asig_Fech.click()
selecb(driver,"Menú para Empresa asignada","América Móvil Perú SAC")
selecb(driver,"Menú para Organización asignada","REGIONAL DEL NORTE")
for i in range(3):
    try:
        selecb(driver,"Menú para Grupo asignado+",Lugar)
        print('Lugar Seleccionado')
        break
    except TimeoutException:
        print('Intento numero: ',i)
        out = driver.find_element(By.XPATH,'//*[@id="WIN_0_10010314"]/table/tbody/tr/td')
        out.click()
selecb(driver,"Menú para Usuario asignado+",oYm)     
## Añadir manejo de error para cuando es formato de fecha 


txtbox_FPI = driver.find_element(By.XPATH,'//*[@id="WIN_0_10000515"]/div/label[contains(text(),"Fecha programada de inicio+")]/following::input[@class="text  realrtl "][1]')
txtbox_FPF = driver.find_element(By.XPATH,'//*[@id="WIN_0_10000515"]/div/label[contains(text(),"Fecha programada de finalización+")]/following::input[@class="text  realrtl "][1]')
# Formato de fecha:  07/02/2024 12:00:00 a.m.
txtbox_FPI.send_keys(FStar)
txtbox_FPF.send_keys(FEnd)

## Relaciones 

Relaciones = wait.until(EC.element_to_be_clickable((By.XPATH,'//*[@id="WIN_0_10000500"]/div[2]/div[2]/div/dl/dd[span[a[contains(text(),"Relaciones")]]]//a')))
Relaciones.click()
#seleciono del menu desplegable
T_registro = wait.until(EC.element_to_be_clickable((By.XPATH,'//*[@id="WIN_0_10000512"]/div/label[contains(text(),"Tipo de registro")]/following::textarea[@class="text sr "][1]')))
T_registro.click()


Elem_cofig = wait.until(EC.presence_of_element_located((By.XPATH,"/html/body/div/div/table/tbody/tr[td[contains(text(),'Elemento de configuración')]]//td")))
Elem_cofig.click()
Relac_bttn = driver.find_element(By.XPATH,"//*[contains(text(), 'Relacionar')]")
Relac_bttn.click() 

## Switch de ventana

swtich_wind(driver,'Búsqueda de relación de CI')

## Busco site 

Site_Code = Site_Code.replace(' ','')
#Cod_Site = 'TAL3878_Chipta'

Site_txbox = wait.until(EC.presence_of_element_located((By.XPATH,"//*[@id='WIN_0_304246250']/div/div/div/fieldset/div/div[label[contains(text(),'Buscar+')]]//textarea")))##
Site_txbox.clear()
Site_txbox.send_keys(Site_Code)
Srch_bttn = driver.find_element(By.XPATH,"//*[@id='WIN_0_304246260']/fieldset/div/a/div[@class='btnimgdiv']")
Srch_bttn.click()                                

a = driver.find_elements(By.XPATH,'//*[@id="T301260800"]/tbody/tr/td[1]')
for elemento in a:
    Name = elemento.text.split('_')[-1]
    if contiene_numero(Name):
        print(elemento.text)
    else: 
        site = elemento.text
        elemento.click()
Relac_bttn = driver.find_element(By.XPATH,"//*[@id='FormContainer']/a/div/div[text() ='Relacionar']")
Relac_bttn.click()   
#WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.XPATH, "//iframe[starts-with(@id, 'id_base_del_iframe')]")))


Iframe = driver.find_element(By.XPATH, "//iframe[starts-with(@id, '17')]") 
driver.switch_to.frame(Iframe)
Aceptar_bttn = driver.find_element(By.XPATH,"/html/body/div/a[text() = 'Aceptar']")
Aceptar_bttn.click()

time.sleep(0.5)
swtich_wind(driver,'Atributo de la relación')
Crr_btt = wait.until(EC.element_to_be_clickable((By.XPATH,"//*[@id='FormApp']/div/a/div/div[text() ='Cerrar']")))
time.sleep(1)
Crr_btt.click()

swtich_wind(driver,'Crear tarea')
Guardar_bttn = wait.until(EC.element_to_be_clickable((By.XPATH,"//*[@id='FormContainer']/a/div/div[text() ='Guardar']")))
Guardar_bttn.click()


Lugar Seleccionado


In [64]:


swtich_wind(driver,'BMC Remedy (Buscar)')

Fecha_sis_bttn = wait.until(EC.element_to_be_clickable((By.XPATH,'//div[2]/div[2]/div/dl/dd/span/a[text()="Fecha/sistema"]')))
Fecha_sis_bttn.click()

VFech_Tar = driver.find_element(By.XPATH,"//a/div/div[text() = 'Ver fecha de tarea']")
time.sleep(1)
VFech_Tar.click()
swtich_wind(driver,'Fechas de la tarea')
bttn_crr = wait.until(EC.element_to_be_clickable((By.XPATH,'//div[@class="f1" and text()="Cerrar"]')))
bttn_crr.click()

swtich_wind(driver,'BMC Remedy (Buscar)')
Rv_bttn = wait.until(EC.element_to_be_clickable((By.XPATH,'//a/div[img[@title="Usar fecha de inicio programada de la tarea"]]')))
Rv_bttn.click()
RR_bttn = wait.until(EC.element_to_be_clickable((By.XPATH,'//a/div[img[@title="Usar fecha de finalización programada de la tarea"]]')))
RR_bttn.click()

Siguiente_f = driver.find_element(By.XPATH,'//fieldset/div/a/div/div[text() = "Siguiente fase"]')
Siguiente_f.click() 


### Implanto

In [65]:
# espera hasta 10 segundos antes de lanzar una excepción
time.sleep(1.5)
Consola_d_Cambios(wait,driver)

#################################################################################
time.sleep(3.5)


################################## Localizo la CRQ x Numero ##############################
for intento in range(0,4):
    try:
        a = wait.until(EC.visibility_of_element_located((By.XPATH,f'//*[@id="T301872400"]/tbody/tr[td[nobr[span[text() = "{CRQ}"]]]]')))
        time.sleep(4)
        a.click()
        a.send_keys(Keys.RETURN)
        print('Encontre CRQ, Implanto')
        break
        
    except:
        print('No se cuentra la CRQ creada: ',CRQ,'Refresco y vuelvo a intentar')
        driver.refresh()
        Consola_d_Cambios(wait,driver)
    
########################################################################################33
#TODO: Para cada etapa que regrese en que etapa va
Today_str = datetime.today().strftime('%d-%m-%y')
a = driver.find_element(By.XPATH,'//*[@id="WIN_0_304248710"]/fieldset/div/dl/dd[5]/span[2]')
display(a.text)
for i in range(0,4):
    Sgt_Fase = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[@class='f7' and  text()='Siguiente fase']")))
    time.sleep(3)
    Sgt_Fase.click()
    time.sleep(2)

Nueva_linea = FStar+',' + site + ','+a.text
display(Nueva_linea)
# Uso de la función
añadir_linea(r'C:\Users\C26764\America Movil Peru S.A.C\EAS - 2\MOP_CRQS\CRQS\CRQS.txt', Nueva_linea)
display(CRQ)
swtich_wind(driver,'Cierre del cambio')
driver.close()
swtich_wind(driver,'BMC Remedy (Buscar)')


Encontre CRQ, Implanto


''

'19-07-2024 08:00:00,RCSRSANTASANTA,CRQ000002863048'

'CRQ000002863048'

In [ ]:
### AGREGA ´PARA CERRAR PESTAÑA QUE SALE LUEGO DE IMPLANTAR

###
-3 click en siguiente 

In [ ]:
def crear_ticket():
    # Aquí va tu código para crear un ticket
    pass

while True:
    crear_ticket()
    respuesta = input("¿Deseas crear otro ticket? (s/n): ")
    if respuesta.lower() != 's':
        break


### Crear archivo Txt Donde se guardara los CRQS